In [1]:
import pandas as pd
import requests
from lxml import html
import requests

In [2]:
words = pd.read_excel('Words from GRE Vocabulary Assessment Videos.xlsx')

In [3]:
set1 = words['Group 1'][1:31]

In [4]:
set1

1           abound
2        amorphous
3          austere
4            belie
5       capricious
6         cerebral
7        congenial
8      conspicuous
9          cursory
10        daunting
11           deify
12        didactic
13     disseminate
14        feasible
15           flout
16     homogeneous
17         humdrum
18         insipid
19      loquacious
20    misanthropic
21        misnomer
22       negligent
23      obsequious
24         placate
25      proclivity
26         puerile
27        quixotic
28     spendthrift
29        taciturn
30            wary
Name: Group 1, dtype: object

In [88]:
word = 'obsequious'

In [136]:
tree.xpath('//div[@class="card-text"]/p//text()')[1]

'Powered by '

In [148]:
for word in set1:
    url = 'https://mnemonicdictionary.com/?word='+word

    page = requests.get(url)
    tree = html.fromstring(page.content)
    definition = tree.xpath('//li[@class="media list-group-item p-4"]/div[1]//text()')[12]
    print("\n\nWord: \t\t{}\nDefinition: \t{}".format(word, definition))
    for mnemonic_id in range(0,10):
        try:
            mnemonic = tree.xpath('//div[@class="card-text"]/p//text()')[mnemonic_id].strip()
        except:
            continue
        if mnemonic in ['','Powered by','Mnemonic Dictionary']:
            continue
        print("Mnemonic {}: \t{}".format(mnemonic_id, mnemonic))



Word: 		abound
Definition: 	 (verb) be abundant or plentiful; exist in large quantities
                      
                  
Mnemonic 0: 	Bounded means limited. Abound means not limited, that is plentiful.
Mnemonic 1: 	Abound sounds like abundant which means in plenty...so ABOUND means to exist in large number


Word: 		amorphous
Definition: 	 (adj) having no definite form or distinct shape
                              
Mnemonic 3: 	In Greek 'morphe' means shape/form.
So, amorphous(a +  morph + ous) means 'without shape/shapeless'.
Similarly 'morphing'  means shaping(the original).
'Metamorphosis' = Change in form/nature
Mnemonic 4: 	amorphous = a[morf]us = on reversing morf become --> a+form+us i.e something without form.. or something lacking shape..
Mnemonic 5: 	Alas! I can't creat any mnemonic for this word! Because the word 'Amorphous' has no distinct shape.
Mnemonic 6: 	morph= change shape
amorphous = no shape
Mnemonic 7: 	amorphous-a+morp(marpu)+us....in telugu marpu mea

In [114]:
for i in [0,5,6]:
    print(tree.xpath('//div[@class="card-text"]/p//text()')[i].strip())

or obse(ssed)-Qui( yes in frech)-ous=always obsessed with saying yes to it all
seq = Suck, uio = Your, us = Ass. He sucks your ass, therefore he's obsequious.
Think "seek" because when you seek something you show attentiveness to find it.


In [ ]:
<div class="card-text">
           <p>

In [85]:
for word in set1:
    url = 'https://mnemonicdictionary.com/?word='+word
    
    page = requests.get(url)
    tree = html.fromstring(page.content)
    definition = tree.xpath('//li[@class="media list-group-item p-4"]/div[1]//text()')[12]
    print(word, definition)

abound  (verb) be abundant or plentiful; exist in large quantities
                      
                  
amorphous  (adj) having no definite form or distinct shape
                              
austere  (adj) severely simple
                              
belie  (verb) be in contradiction with
                              
capricious  (adj) changeable
                              
cerebral  (adj) involving intelligence rather than emotions or instinct
                              
congenial  (adj) suitable to your needs
                               
conspicuous  (adj) obvious to the eye or mind
                               
cursory  (adj) hasty and without attention to detail; not thorough
                              
daunting  (adj) discouraging through fear
                              
deify  (verb) consider as a god or godlike
                               
didactic  (adj) instructive (especially excessively)
                              
disseminate  (verb) cause 

obsequious  (adj) attempting to win favor from influential people by flattery
                              


In [ ]:
media list-group-item p-4

In [ ]:
#This will create a list of buyers:
buyers = tree.xpath('//div[@title="buyer-name"]/text()')
#This will create a list of prices
prices = tree.xpath('//span[@class="item-price"]/text()')

In [8]:
page = requests.get(url)
page.status_code

200

In [12]:
from bs4 import BeautifulSoup

In [13]:
soup = BeautifulSoup(page.text, 'html.parser')

In [17]:
soup.find_all('p')

[<p>
           obsequious - Dictionary definition and meaning for word 
           <strong>obsequious</strong></p>,
 <p>
             or obse(ssed)-Qui( yes in frech)-ous=always obsessed with saying yes to it all
             </p>,
 <p>
             ob + SEQUI + ous .. sequi means sequence where one thing "follows" the other.. and servants follow what their masters say.. 
             </p>,
 <p>
             seq = Suck, uio = Your, us = Ass. He sucks your ass, therefore he's obsequious.
             </p>,
 <p>
             Think "seek" because when you seek something you show attentiveness to find it. 
             </p>,
 <p>
             sounds like AP+SEEK+QUOUS.......when you seek help,you are attentive and your manner is servile
             </p>,
 <p class="md-attribution">Powered by <a href="https://mnemonicdictionary.com">Mnemonic Dictionary</a>
 </p>,
 <p>
             obsequious - obey sequence of work in office(obedience, dutiful).
             </p>,
 <p>
             Sounds

In [28]:
soup.find_all('p')[2].get_text()

'\n            ob + SEQUI + ous .. sequi means sequence where one thing "follows" the other.. and servants follow what their masters say.. \n            '

In [17]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="MnemonicDictionary.com - Meaning of obsequious and a memory aid (called Mnemonic) to retain that meaning for long time in our memory." name="description"/>
  <meta content="obsequious, meaning, definition, word power made easy, mnemonic, mnemonics, dictionary, gre words, gre dictionary, vocabulary, word list" name="keywords"/>
  <meta content="OpenCubicles" name="author">
   <title>
    obsequious meaning - definition of obsequious by Mnemonic Dictionary
   </title>
   <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,300,600" rel="stylesheet" type="text/css"/>
   <link href="/assets/css/toolkit.css" rel="stylesheet"/>
   <link href="/assets/css/application.css" rel="stylesheet"/>
   <link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.0.